In [9]:
import pandas as pd
import re
import nltk
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.feature_extraction.text import CountVectorizer
# 加载python库
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [4]:
# 数据读入
df = pd.read_csv('data/FakeNewsNet.csv')
data = df[['title', 'real']]

In [5]:
# 分词
news_text = []
for content in data['title']:
    content = re.sub('[^\w ]','',content)
    content = nltk.word_tokenize(content)
    content = " ".join(content)
    news_text.append(content)
print(news_text[0])

Kandi Burruss Explodes Over Rape Accusation on Real Housewives of Atlanta Reunion Video


In [6]:
# 向量化
vec_text = CountVectorizer(stop_words="english", max_features=1000)
text_matrix = vec_text.fit_transform(news_text).toarray()

print(text_matrix.shape)

(23196, 1000)


In [12]:
Y_train = np.array(data['real'])
X_train = text_matrix
print(X_train.shape)
print(Y_train.shape)

(23196, 1000)
(23196,)


In [15]:
class ANN(nn.Module):

    def __init__(self):

        super(ANN,self).__init__() # Inhertiting

        # self.linear1 = nn.Linear(1000,500) # IN 5008 OUT 2000
        # self.relu1 = nn.ReLU() # Actfunc 1

        self.linear2 = nn.Linear(1000,500) # IN 2000 OUT 500
        self.relu2 = nn.ReLU()

        self.linear3 = nn.Linear(500,100) # IN 500 OUT 100
        self.relu3 = nn.ReLU()

        self.linear4 = nn.Linear(100,20) # IN 100 OUT 20
        self.relu4 = nn.ReLU()

        self.linear5 = nn.Linear(20,2) # IN 20 OUT 2 | OUTPUT


    def forward(self,x):

        # out = self.linear1(x) # Input Layer
        # out = self.relu1(out)

        out = self.linear2(x) # Hidden Layer 1
        out = self.relu2(out)

        out = self.linear3(out) # Hidden Layer 2
        out = self.relu3(out)

        out = self.linear4(out) # Hidden Layer 3
        out = self.relu4(out)


        out = self.linear5(out) # Output Layer

        return out

model = ANN()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
error = nn.CrossEntropyLoss()

In [17]:
# Converting numpy arrays into pytorch tensors
X_train = torch.Tensor(X_train)

# You must convert it into LongTensor. I did it once
Y_train = torch.Tensor(Y_train).type(torch.LongTensor)

# X_test = torch.Tensor(X_test)
# Y_test = torch.Tensor(Y_test)

EPOCHS = 20

for epoch in range(EPOCHS):

    # Clearing gradients
    optimizer.zero_grad()

    # Forward Propagation
    outs = model(X_train)

    # Computing Loss
    loss = error(outs,Y_train)

    # Backward propagation
    loss.backward()

    # Updating parameters
    optimizer.step()

    # Printing loss
    print(f"Loss after iteration {epoch} is {loss}")

Loss after iteration 0 is 0.4137013256549835
Loss after iteration 1 is 0.40739601850509644
Loss after iteration 2 is 0.4051819443702698
Loss after iteration 3 is 0.4013769030570984
Loss after iteration 4 is 0.3939294219017029
Loss after iteration 5 is 0.3863558769226074
Loss after iteration 6 is 0.3799715042114258
Loss after iteration 7 is 0.3735673725605011
Loss after iteration 8 is 0.36670076847076416
Loss after iteration 9 is 0.3595882058143616
Loss after iteration 10 is 0.35128575563430786
Loss after iteration 11 is 0.34165263175964355
Loss after iteration 12 is 0.33131590485572815
Loss after iteration 13 is 0.3199518918991089
Loss after iteration 14 is 0.3069359064102173
Loss after iteration 15 is 0.29236018657684326
Loss after iteration 16 is 0.27591249346733093
Loss after iteration 17 is 0.257546603679657
Loss after iteration 18 is 0.23754224181175232
Loss after iteration 19 is 0.21663156151771545


In [25]:
import csv
filename = 'data/res.txt'
with open(filename,'r') as f:
    reader = csv.reader(f)
    for row in reader:
        print(row)


['0', '4', '6', '8', '11', '13', '14', '15', '16', '26', '27', '30', '31', '32', '35', '36', '40', '44', '45', '51', '53', '54', '55', '56', '57', '62', '66', '67', '68', '70', '72', '74', '75', '77', '78', '81', '83', '87', '89', '97', '102', '105', '113', '114', '119', '120', '123', '126', '127', '129', '132', '135', '136', '139', '140', '144', '146', '148', '150', '152', '154', '157', '158', '160', '161', '163', '164', '168', '173', '180', '185', '186', '192', '195', '197', '198', '199', '205', '207', '209', '211', '214', '215', '216', '223', '224', '226', '229', '234', '236', '240', '242', '243', '245', '247', '249', '250', '252', '253', '254', '256', '257', '258', '259', '260', '261', '263', '265', '267', '268', '269', '276', '280', '284', '299', '303', '304', '305', '307', '308', '311', '312', '313', '315', '316', '317', '320', '321', '323', '328', '330', '331', '332', '333', '336', '337', '338', '341', '342', '343', '344', '347', '354', '355', '357', '358', '359', '361', '364', 

In [21]:
answer = pd.read_csv('data/answer6.csv',usecols=['id','answer'])
answer

,id,answer
0,0,REAL
1,1,REAL
2,2,REAL
3,3,REAL
4,4,REAL
...,...,...
1262,1262,FAKE
1263,1263,FAKE
1264,1264,REAL
1265,1265,FAKE


In [32]:
for i in row:
    if answer.iloc[int(i),1] == 'REAL':
        answer.iloc[int(i),1] = 'FAKE'
    else:
        answer.iloc[int(i),1] = 'REAL'

In [33]:
answer

,id,answer
0,0,FAKE
1,1,REAL
2,2,REAL
3,3,REAL
4,4,FAKE
...,...,...
1262,1262,REAL
1263,1263,FAKE
1264,1264,FAKE
1265,1265,REAL


In [34]:
answer.to_csv('data/true_answer.csv', index=False,  encoding='utf8')